### ロジスティクス・ネットワーク設計問題
問題：需要を満たしつつ輸送費と生産費の和が最小となるように、どこで何をどれだけ生産し、どう輸送するかを求める

#### 入力データの作成
2種類A,Bの製品を2つの工場X,Yで生産し、2箇所の需要地P,Qへ輸送する

In [2]:
prod=list('AB')
dem=list('PQ')
man=list('XY')
lane=(2,2)

In [3]:
import numpy as np,pandas as pd
tbdi=pd.DataFrame(((j,k) for j in dem for k in man),
                  columns=['需要地','工場'])
tbdi['輸送費']=[1,2,3,1]
tbdi

,需要地,工場,輸送費
0,P,X,1
1,P,Y,2
2,Q,X,3
3,Q,Y,1


In [4]:
tbde=pd.DataFrame(((j,i) for j in dem for i in prod),
                  columns=['需要地','製品'])
tbde['需要']=[10,10,20,20]
tbde

,需要地,製品,需要
0,P,A,10
1,P,B,10
2,Q,A,20
3,Q,B,20


In [6]:
tbfa=pd.DataFrame(((k,l,i,0,np.inf)
                   for k,nl in zip(man,lane)
                   for l in range(nl)
                   for i in prod),
                  columns=['工場','レーン','製品','下限','上限'])
tbfa['生産費']=[1,np.nan,np.nan,1,3,np.nan,5,3]
tbfa.dropna(inplace=True)
tbfa.loc[4,'上限']=10
tbfa

,工場,レーン,製品,下限,上限,生産費
0,X,0,A,0,inf,1.0
3,X,1,B,0,inf,1.0
4,Y,0,A,0,10.0,3.0
6,Y,1,A,0,inf,5.0
7,Y,1,B,0,inf,3.0


#### 作成した3つの表をlogistics_networkに渡し、実行する

In [10]:
from ortoolpy import logistics_network
_,tbdi2,_=logistics_network(tbde,tbdi,tbfa)

In [11]:
tbfa[tbfa.ValY >0]

,工場,レーン,製品,下限,上限,生産費,VarY,ValY
0,X,0,A,0,inf,1.0,v000009,20.0
3,X,1,B,0,inf,1.0,v000010,10.0
4,Y,0,A,0,10.0,3.0,v000011,10.0
7,Y,1,B,0,inf,3.0,v000013,20.0


In [12]:
tbdi2[tbdi2.ValX >0]

,需要地,工場,輸送費,製品,VarX,ValX
0,P,X,1,A,v000001,10.0
1,P,X,1,B,v000002,10.0
2,Q,X,3,A,v000003,10.0
6,Q,Y,1,A,v000007,10.0
7,Q,Y,1,B,v000008,20.0
